## Discipline of AI Checkpoint 01
develop study of data of post's twitter's influence in price of BTC
### Step one - Pre-processing of data
- 


In [22]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

#For Preprocessing
import re    # RegEx for removing non-letter characters
import nltk  # natural language processing
nltk.download("stopwords")
nltk.downloader.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# For Building the model
from sklearn.model_selection import train_test_split
import tensorflow as tf
import seaborn as sns

#For data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline


pd.options.plotting.backend = "plotly"


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joelm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\joelm\AppData\Roaming\nltk_data...


# Load Tweet dataset
dataset price bitcoin: https://www.investing.com/crypto/bitcoin/historical-data
dataset twitter: https://www.kaggle.com/datasets/kaushiksuresh147/bitcoin-tweets

In [67]:
# Load Tweet dataset
df1 = pd.read_csv('../data/raw/Bitcoin_tweets.csv')
df2 = pd.read_csv('../data/raw/BitcoinPriceHistory.csv')

df2['Date'] = pd.to_datetime(df2['Date'], format='%m/%d/%Y')
df2['Date'] = df2['Date'].dt.tz_localize('UTC').dt.strftime('%m-%d-%Y')

df1['date'] = pd.to_datetime(df1['date'])
df1['date'] = df1['date'].dt.tz_localize('UTC').dt.strftime('%m-%d-%Y')

df1['price'] = df1['date'].map(df2.set_index('Date')['Price'])
# Validate if the data equals a certain value
validation_value = 'expected_value'
mask = df1['price'] == validation_value

if mask.all():
    print('Data equals the expected value')
else:
    print('Data does not equal the expected value')

df1[['date', 'price', 'text']]



df = df1[['text']][:20000]
df


MemoryError: Unable to allocate 512. KiB for an array with shape (65536,) and data type object

In [3]:
def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

In [4]:
cleantext=[]
for item in tqdm(df['text']):
    words=tweet_to_words(item)
    cleantext+=[words]
df['cleantext']=cleantext
df

100%|██████████| 20000/20000 [00:54<00:00, 367.27it/s]


,text,cleantext
0,Blue Ridge Bank shares halted by NYSE after #b...,"[blue, ridg, bank, share, halt, nyse, bitcoin,..."
1,"😎 Today, that's this #Thursday, we will do a ""...","[today, thursday, take, 2, friend, leowandersl..."
2,"Guys evening, I have read this article about B...","[guy, even, read, articl, btc, would, like, sh..."
3,$BTC A big chance in a billion! Price: \487264...,"[btc, big, chanc, billion, price, 4872644, 0, ..."
4,This network is secured by 9 508 nodes as of t...,"[network, secur, 9, 508, node, today, soon, bi..."
...,...,...
19995,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...,"[btc, sell, pressur, alert, price, trade, arou..."
19996,@HusseinChindo @kayodebakre8 Wrong! #Bitcoin i...,"[husseinchindo, kayodebakre8, wrong, bitcoin, ..."
19997,LINK IN MY BIO!!! \nFlex is dropping officiall...,"[link, bio, flex, drop, offici, tomorrow, feb,..."
19998,#Bitcoin #Btc\n\nWait and see.... https://t.co...,"[bitcoin, btc, wait, see, http, co, vdaseguhlp]"


In [18]:
def unlist(list):
    return ''.join(f'{item} ' for item in list)

In [25]:
def compute_vader_scores(df, label):
    sid = SentimentIntensityAnalyzer()
    df["negative"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neg"])
    df["neutral"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neu"])
    df["positive"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["pos"])
    df["multi_sentimental"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["compound"])
    df['cleantext2'] = df[label].apply(lambda x: unlist(x))
    return df

In [29]:
df2 = compute_vader_scores(df,'cleantext')
df2[['cleantext', 'cleantext2', 'positive', 'negative', 'neutral', 'multi_sentimental']]

,cleantext,cleantext2,positive,negative,neutral,multi_sentimental
0,"[blue, ridg, bank, share, halt, nyse, bitcoin,...",blue ridg bank share halt nyse bitcoin atm ann...,0.128,0.000,0.872,0.2960
1,"[today, thursday, take, 2, friend, leowandersl...",today thursday take 2 friend leowandersleb btc...,0.225,0.000,0.775,0.4939
2,"[guy, even, read, articl, btc, would, like, sh...",guy even read articl btc would like share http...,0.281,0.000,0.719,0.5719
3,"[btc, big, chanc, billion, price, 4872644, 0, ...",btc big chanc billion price 4872644 0 2021 02 ...,0.000,0.000,1.000,0.0000
4,"[network, secur, 9, 508, node, today, soon, bi...",network secur 9 508 node today soon biggest be...,0.000,0.200,0.800,-0.5423
...,...,...,...,...,...,...
19995,"[btc, sell, pressur, alert, price, trade, arou...",btc sell pressur alert price trade around 3782...,0.145,0.000,0.855,0.2960
19996,"[husseinchindo, kayodebakre8, wrong, bitcoin, ...",husseinchindo kayodebakre8 wrong bitcoin legal...,0.197,0.165,0.638,-0.1027
19997,"[link, bio, flex, drop, offici, tomorrow, feb,...",link bio flex drop offici tomorrow feb 06 2021...,0.000,0.130,0.870,-0.2732
19998,"[bitcoin, btc, wait, see, http, co, vdaseguhlp]",bitcoin btc wait see http co vdaseguhlp,0.000,0.000,1.000,0.0000
